In [ ]:
from nbpypath import *

In [ ]:
import numpy as np
import pandas as pd

from numba import njit


In [ ]:
from oscb import *

# customize Bokeh plotting in the Jupyter Notebook environment
setup_notebook()

In [ ]:
# Seed the PRNG
random_seed(12345)

In [ ]:
lnet = LetterNet(
    MAX_SYNAPSES=1_000_000,  # max number of synapses, one million by default
    N_COLS_PER_LETTER=10,  # distributedness of letter SDR
    SPARSE_FACTOR=5,  # sparseness of letter SDR
    N_CELLS_PER_COL=100,  # per mini-column capacity
)

In [ ]:
%%time

lnet.learn_words_as_sequence(
    [], # no input data to see JIT time cost
    sp_width=(1, 1),  # width of spike train: [n_columns, n_cells]
    sp_thick=1,  # thickness of spike train
)


In [ ]:
%%time

lnet.learn_words_as_sequence(
    '''
the quick brown fox jumps over the old lazy dog
the quick red   fox jumps over the old lazy dog
''',
    sp_width=(3, 20),  # width of spike train: [n_columns, n_cells]
    sp_thick=60,  # thickness of spike train
    
    link_back=True,  # create backward synapses too
)


In [ ]:
# get a DataFrame about all excitatory synapses so far
excit_df = lnet.excitatory_synapses()
excit_df

In [ ]:
lsim = LetterNetSim(
    lnet,

    # global scaling factor, to accommodate a unit synaptic efficacy value of 1.0
    # roughly this specifies that:
    #   how many presynaptic spikes is enough to trigger a postsynaptic spike,
    #   when each synapse has a unit efficacy value of 1.0
    SYNAP_FACTOR=30,

    # reset voltage, equal to VOLT_REST, or lower to enable refractory period
    VOLT_RESET=-0.1,
    # membrane time constant
    τ_m=10,

    # fire plot params
    plot_width=800,
    plot_height=600,
    plot_n_steps=60,
    fire_dots_glyph="square",
    fire_dots_alpha=0.01,
    fire_dots_size=3,
    fire_dots_color="#0000FF",
)

lsim.simulate(
    50, # n steps

    '''
quick
''',
    prompt_blur=1.0,  # don NOT suppress other signals
)

# ref the learnt seq:
#   the quick brown fox jumps over the old lazy dog
show(lsim.fig)

In [ ]:
# create 2M inhibitory synapses randomly
lnet.create_inhibitory_links_randomly( 2_000_000 )

# get a DataFrame about all inhibitory synapses so far
inhib_df = lnet.inhibitory_synapses()
inhib_df

In [ ]:
# create 2M excitatory synapses randomly
lnet.create_excitatory_links_randomly( 2_000_000 )

# get a DataFrame about all excitatory synapses so far
excit_df = lnet.excitatory_synapses()
excit_df

In [ ]:
# Simulate with those random links created
lsim = LetterNetSim(
    lnet,

    # global scaling factor, to accommodate a unit synaptic efficacy value of 1.0
    # roughly this specifies that:
    #   how many presynaptic spikes is enough to trigger a postsynaptic spike,
    #   when each synapse has a unit efficacy value of 1.0
    SYNAP_FACTOR=30,

    # reset voltage, equal to VOLT_REST, or lower to enable refractory period
    VOLT_RESET=-0.1,
    # membrane time constant
    τ_m=10,

    # fire plot params
    plot_width=800,
    plot_height=600,
    plot_n_steps=60,
    fire_dots_glyph="square",
    fire_dots_alpha=0.01,
    fire_dots_size=3,
    fire_dots_color="#0000FF",
)

lsim.simulate(
    50, # n steps

    '''
quick
''',
    prompt_blur=1.0,  # don NOT suppress other signals
)

# ref the learnt seq:
#   the quick brown fox jumps over the old lazy dog
show(lsim.fig)